# Common Use Cases when dealing with Hypothesis Testing

In [146]:
import random
import pandas as pd
import numpy as np

## USE-CASE: Testing Proportions 

### Is coin biased ?
- We toss coin 250 times, 140 heads, 120 tails. 

In [197]:
# se have:
n_h = 140
n_t = 110
observations = (n_h, n_t)
n_observations = n_h + n_t
print observations, n_observations, 

(140, 110) 250


In [198]:
# We define the null hypothesis and the test statistic

def run_null_hypothesis(n_observations):
    """the model of Null hypothesis"""
    sample = [random.choice('HT') for _ in range(n_observations)]
    df = pd.DataFrame(sample)
    value_counts = df[0].value_counts()
    n_heads = value_counts['H']
    n_tails = value_counts['T']
    return (n_heads, n_tails)

def test_statistic((n_heads, n_tails)):
    """Computes the test statistic"""
    return abs(n_heads - n_tails)

In [199]:
test_stat_H0 = test_statistic(run_null_hypothesis(n_observations))
test_stat_H1 = test_statistic(observations)

print "Test Statistic for Null Hypothesis H0:",  test_stat_H0
print "Test Statistic for Hypothesis H1:",  test_stat_H1

Test Statistic for Null Hypothesis H0: 18
Test Statistic for Hypothesis H1: 30


In [200]:
# we perform iterations for good statistics
N_ITER = 1000
test_stat_H0_v = [test_statistic(run_null_hypothesis(n_observations)) for _ in range(N_ITER)]

p_value = 1. * sum([1 for test_stat_H0 in test_stat_H0_v if test_stat_H0 >= test_stat_H1])/N_ITER
print "The p-value is: ", p_value

The p-value is:  0.068


### Is dice crooked ?
- we have the frequencies {1:8, 2:9, 3:19, 4:5, 6:8, 6:11}

In [201]:
observations = {1:8, 2:9, 3:19, 4:5, 5:8, 6:11}

observations_frequencies = np.array(observations.values())
n_dice_drops = np.sum(observations_frequencies)
print n_dice_drops

60


In [202]:
def run_null_hypothesis(n_dice_drops):
    """the model of Null hypothesis"""
    dice_values = [1, 2, 3, 4, 5, 6]
    rolls = np.random.choice(dice_values, n_dice_drops, replace=True)
    return np.array(dict(pd.DataFrame(rolls)[0].value_counts()).values())

def test_statistic(dice_frequencies, n_dice_drops):
    """Computes the test statistic"""
    expected_frequencies = np.ones(6) * n_dice_drops / 6.
    return sum(abs(dice_frequencies - expected_frequencies))

In [203]:
test_stat_H0 = test_statistic(run_null_hypothesis(n_dice_drops), n_dice_drops)
test_stat_H1 = test_statistic(observations_frequencies, n_dice_drops)

print "Test Statistic for Null Hypothesis H0:",  test_stat_H0
print "Test Statistic for Hypothesis H1:",  test_stat_H1


Test Statistic for Null Hypothesis H0: 16.0
Test Statistic for Hypothesis H1: 20.0


In [207]:
# we perform iterations for good statistics
N_ITER = 1000
test_stat_H0_v = [test_statistic(run_null_hypothesis(n_dice_drops), n_dice_drops) for _ in range(N_ITER)]

p_value = 1. * sum([1 for test_stat_H0 in test_stat_H0_v if test_stat_H0 >= test_stat_H1])/N_ITER
print "The p-value is: ", p_value

The p-value is:  0.135
